In [ ]:
!pip install ultralytics

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [7]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks



Ultralytics YOLOv8.2.95 🚀 Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Setup complete ✅ (12 CPUs, 31.9 GB RAM, 361.0/475.6 GB disk)

OS                  Windows-11-10.0.22631-SP0
Environment         Windows
Python              3.12.4
Install             pip
RAM                 31.91 GB
CPU                 Intel Core(TM) i7-10750H 2.60GHz
CUDA                11.8

numpy               ✅ 1.26.4<2.0.0,>=1.23.0
matplotlib          ✅ 3.8.4>=3.3.0
opencv-python       ✅ 4.10.0.84>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.32.2>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.4.1+cu118>=1.8.0
torchvision         ✅ 0.19.1+cu118>=0.9.0
tqdm                ✅ 4.66.4>=4.64.0
psutil              ✅ 5.9.0
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.2.2>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0
ultralytics-thop    ✅ 2.0.6>=2.0.0
torch               ✅ 2.4.1+cu118!=2.4.0,>=1.8.0; sys_p

### Process KangaiSet+Manga109

In [11]:
import os
import pandas as pd
import shutil
import random

# make train, val, test dirs
kangai_yolo_dir = 'KangaiSet_YOLO'
for dataset_type in ['train', 'val', 'test']:
  os.makedirs(f'../{kangai_yolo_dir}/{dataset_type}/images', exist_ok=True)
  os.makedirs(f'../{kangai_yolo_dir}/{dataset_type}/labels', exist_ok=True)

width = 1654
height = 1170
emotion_map = {
    'anger': 0,
    'disgust': 1,
    'fear': 2,
    'happiness': 3,
    'neutral': 4,
    'sadness': 5,
    'surprise': 6
}

root_dir = '../Manga109'
images_dir = os.path.join(root_dir, 'images')
annotations_dir = os.path.join(root_dir, 'annotations')

df = pd.read_csv('../Manga109/kangaiset_annotation_long.csv')
df = df.sort_values(by=['page_path']) # sort df by page_path for ease of annotation

count_row = df.shape[0]  # Gives number of rows

# go through kangaiset_annotation_long.csv
for index, row in df.iterrows():
  
  manga_name = row['manga_name']
  page_path = row['page_path']

  book_page = page_path.strip('images/')[:-3].replace('/','_')
  page_num = book_page[len(book_page)-3:]
  # print(book_page, page_num)

  # randomly assign to train/val/test and copy image + txt file to new dir
  dataset_type = random.choices(['train', 'val', 'test'], weights=[0.85, 0.10, 0.05])[0]

  src_image_path = os.path.join(images_dir, f'{manga_name}/{page_num}.jpg')
  yolo_image_path = os.path.join(f'../{kangai_yolo_dir}/{dataset_type}/images/{book_page}.jpg')

  # copy image from Manga109 to KangaiSet_YOLO
  shutil.copyfile(src_image_path, yolo_image_path)

  face_xmin = row['face_xmin']	
  face_ymin = row['face_ymin']	
  face_xmax = row['face_xmax']	
  face_ymax = row['face_ymax']

  # emotion_str = row['emotion_str']
  # emotions = ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

  label_file_path = f'../{kangai_yolo_dir}/{dataset_type}/labels/{book_page}.txt'
  # label_file = open(label_file_path, 'a')
  with open(label_file_path, 'a') as label_file:
    if face_xmin < face_xmax and face_ymin < face_ymax:
      bbox = ((face_xmin + face_xmax) / (2 * width),
              (face_ymin + face_ymax) / (2 * height),
              (face_xmax - face_xmin) / width,
              (face_ymax - face_ymin) / height,
              )
      # label_file.write(f'{emotions.index(emotion_str)} {" ".join(map(str, bbox))}\n')
      label_file.write(f'{emotion_map[emotion_str]} {" ".join(map(str, bbox))}\n')
  # label_file.close()

  count_row -= 1
  if count_row > 0:
    if count_row % 50 == 0:
      print(f'{count_row} images left to process')
  else:
    print('Processing complete!')



9350 images left to process
9300 images left to process
9250 images left to process
9200 images left to process
9150 images left to process
9100 images left to process
9050 images left to process
9000 images left to process
8950 images left to process
8900 images left to process
8850 images left to process
8800 images left to process
8750 images left to process
8700 images left to process
8650 images left to process
8600 images left to process
8550 images left to process
8500 images left to process
8450 images left to process
8400 images left to process
8350 images left to process
8300 images left to process
8250 images left to process
8200 images left to process
8150 images left to process
8100 images left to process
8050 images left to process
8000 images left to process
7950 images left to process
7900 images left to process
7850 images left to process
7800 images left to process
7750 images left to process
7700 images left to process
7650 images left to process
7600 images left to 

### Train

In [13]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
!yolo task=detect mode=train model=yolov8n.pt data=kangaiset.yaml lr0=0.001 epochs=50 batch=16 device=cuda dropout=0.5 val=True save=True

In [ ]:
# RE-TRAIN
!yolo task=detect mode=train model='runs/detect/train2/weights/best.pt' data=kangaiset.yaml epochs=50 batch=16 device=cuda dropout=0.5 val=True save=True

### Predict

In [2]:
# test predict w/ best.pt
!yolo predict model='runs/detect/train2/weights/best.pt' source='test_page.jpeg' device=cuda

Ultralytics YOLOv8.2.95 🚀 Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 c:\Users\Zed\Documents\Code\__Senior Design\Senior-Design\bounder_YOLO\Akuhamu_005.jpg: 480x640 (no detections), 26.6ms
Speed: 2.5ms preprocess, 26.6ms inference, 18.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict3
💡 Learn more at https://docs.ultralytics.com/modes/predict
